In [10]:
from google.colab import files
files.upload()


Saving kaggle.json to kaggle (1).json


{'kaggle (1).json': b'{"username":"pavanitejasri","key":"3b930f82b8d8a0892685a397a2049dcd"}'}

In [11]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d leftin/fruit-ripeness-unripe-ripe-and-rotten
!unzip fruit-ripeness-unripe-ripe-and-rotten.zip -d fruit_dataset


Dataset URL: https://www.kaggle.com/datasets/leftin/fruit-ripeness-unripe-ripe-and-rotten
License(s): CC-BY-SA-4.0
fruit-ripeness-unripe-ripe-and-rotten.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  fruit-ripeness-unripe-ripe-and-rotten.zip
replace fruit_dataset/archive (1)/dataset/dataset/test/freshapples/Screen Shot 2018-06-08 at 4.59.44 PM.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [13]:
pip install tensorflow numpy matplotlib opencv-python


In [14]:
!ls



 best_model.h5	       fruit-ripeness-unripe-ripe-and-rotten.zip   kaggle.json
 fruit_dataset	       fruit_shelf_life_model.h5		   sample_data
 fruit_dataset_clean  'kaggle (1).json'


In [15]:
!rm -rf fruit_dataset/'archive (1)'


In [16]:
print(train_gen.class_indices)


{'freshapples': 0, 'freshbanana': 1, 'freshoranges': 2, 'rottenapples': 3, 'rottenbanana': 4, 'rottenoranges': 5, 'unripe apple': 6, 'unripe banana': 7, 'unripe orange': 8}


In [17]:
!unzip -q "fruit-ripeness-unripe-ripe-and-rotten.zip" -d "fruit_dataset"


In [ ]:
!ls -R fruit_dataset


Streaming output truncated to the last 5000 lines.
'rotated_by_15_Screen Shot 2018-06-12 at 8.57.34 PM.png'
'rotated_by_15_Screen Shot 2018-06-12 at 8.57.46 PM.png'
'rotated_by_15_Screen Shot 2018-06-12 at 8.57.54 PM.png'
'rotated_by_15_Screen Shot 2018-06-12 at 8.58.01 PM.png'
'rotated_by_15_Screen Shot 2018-06-12 at 8.58.07 PM.png'
'rotated_by_15_Screen Shot 2018-06-12 at 8.58.18 PM.png'
'rotated_by_15_Screen Shot 2018-06-12 at 8.58.33 PM.png'
'rotated_by_15_Screen Shot 2018-06-12 at 8.58.38 PM.png'
'rotated_by_15_Screen Shot 2018-06-12 at 8.58.43 PM.png'
'rotated_by_15_Screen Shot 2018-06-12 at 8.58.49 PM.png'
'rotated_by_15_Screen Shot 2018-06-12 at 8.58.53 PM.png'
'rotated_by_15_Screen Shot 2018-06-12 at 8.58.57 PM.png'
'rotated_by_15_Screen Shot 2018-06-12 at 8.59.02 PM.png'
'rotated_by_15_Screen Shot 2018-06-12 at 8.59.44 PM.png'
'rotated_by_15_Screen Shot 2018-06-12 at 8.59.57 PM.png'
'rotated_by_15_Screen Shot 2018-06-12 at 9.00.11 PM.png'
'rotated_by_15_Screen Shot 2018-06-12

In [18]:
!mkdir -p fruit_dataset/ripe
!mkdir -p fruit_dataset/unripe
!mkdir -p fruit_dataset/rotten


In [19]:
!find fruit_dataset -type f | head -20



fruit_dataset/archive (1)/dataset/dataset/train/rottenoranges/rotated_by_60_Screen Shot 2018-06-12 at 11.20.30 PM.png
fruit_dataset/archive (1)/dataset/dataset/train/rottenoranges/rotated_by_45_Screen Shot 2018-06-12 at 11.33.55 PM.png
fruit_dataset/archive (1)/dataset/dataset/train/rottenoranges/vertical_flip_Screen Shot 2018-06-12 at 11.37.36 PM.png
fruit_dataset/archive (1)/dataset/dataset/train/rottenoranges/vertical_flip_Screen Shot 2018-06-12 at 11.32.09 PM.png
fruit_dataset/archive (1)/dataset/dataset/train/rottenoranges/rotated_by_15_Screen Shot 2018-06-12 at 11.40.23 PM.png
fruit_dataset/archive (1)/dataset/dataset/train/rottenoranges/vertical_flip_Screen Shot 2018-06-12 at 11.29.58 PM.png
fruit_dataset/archive (1)/dataset/dataset/train/rottenoranges/vertical_flip_Screen Shot 2018-06-12 at 11.21.05 PM.png
fruit_dataset/archive (1)/dataset/dataset/train/rottenoranges/rotated_by_30_Screen Shot 2018-06-12 at 11.25.07 PM.png
fruit_dataset/archive (1)/dataset/dataset/train/rottenor

In [20]:
!mkdir -p fruit_dataset_clean
!cp -r "fruit_dataset/archive (1)/dataset/dataset/train/"* fruit_dataset_clean/


In [21]:
!ls fruit_dataset_clean


 freshapples   freshoranges   rottenbanana   'unripe apple'   'unripe orange'
 freshbanana   rottenapples   rottenoranges  'unripe banana'


In [22]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Data prep
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_gen = datagen.flow_from_directory(
    'fruit_dataset_clean',
    target_size=(128, 128),
    batch_size=32,
    class_mode='sparse',
    subset='training'
)

val_gen = datagen.flow_from_directory(
    'fruit_dataset_clean',
    target_size=(128, 128),
    batch_size=32,
    class_mode='sparse',
    subset='validation'
)


Found 13667 images belonging to 9 classes.
Found 3412 images belonging to 9 classes.


In [23]:
shelf_life_mapping = {
    'unripe apple': 7,
    'freshapples': 4,
    'rottenapples': 0,
    'unripe banana': 5,
    'freshbanana': 3,
    'rottenbanana': 0,
    'unripe orange': 10,
    'freshoranges': 6,
    'rottenoranges': 0
}


In [24]:
import os
import shutil
from sklearn.model_selection import train_test_split

# Path to your train dataset
dataset_path = 'fruit_dataset/archive (1)/dataset/dataset/train'

# Get all the class directories (folders)
class_dirs = os.listdir(dataset_path)

# Create the validation directory if it doesn't exist
val_dir = 'fruit_dataset/archive (1)/dataset/dataset/validation'
os.makedirs(val_dir, exist_ok=True)

# Loop through each class directory to create a validation set
for class_dir in class_dirs:
    class_path = os.path.join(dataset_path, class_dir)

    # Only proceed if it is a directory
    if os.path.isdir(class_path):
        # Get all image files in the class directory
        image_files = os.listdir(class_path)

        # Split the files into train and validation (80% train, 20% validation)
        train_files, val_files = train_test_split(image_files, test_size=0.2, random_state=42)

        # Create the class folder in the validation directory
        val_class_dir = os.path.join(val_dir, class_dir)
        os.makedirs(val_class_dir, exist_ok=True)

        # Move validation images to the validation directory
        for val_file in val_files:
            shutil.move(os.path.join(class_path, val_file), os.path.join(val_class_dir, val_file))

print("Dataset split into training and validation sets.")


Dataset split into training and validation sets.


In [25]:
import os

# Path to your training directory
train_dir = 'fruit_dataset/archive (1)/dataset/dataset/train/unripe banana'

# Get list of all images in the directory
image_files = os.listdir(train_dir)

# Check and print paths of all files
for file in image_files:
    file_path = os.path.join(train_dir, file)
    print(f"File: {file_path}")
    if not os.path.exists(file_path):
        print(f"File missing: {file_path}")


File: fruit_dataset/archive (1)/dataset/dataset/train/unripe banana/aug_0_487.jpg
File: fruit_dataset/archive (1)/dataset/dataset/train/unripe banana/aug_0_9133.jpg
File: fruit_dataset/archive (1)/dataset/dataset/train/unripe banana/aug_0_9619.jpg
File: fruit_dataset/archive (1)/dataset/dataset/train/unripe banana/aug_0_6026.jpg
File: fruit_dataset/archive (1)/dataset/dataset/train/unripe banana/aug_0_7602.jpg
File: fruit_dataset/archive (1)/dataset/dataset/train/unripe banana/aug_0_6855.jpg
File: fruit_dataset/archive (1)/dataset/dataset/train/unripe banana/aug_0_1966.jpg
File: fruit_dataset/archive (1)/dataset/dataset/train/unripe banana/aug_0_528.jpg
File: fruit_dataset/archive (1)/dataset/dataset/train/unripe banana/aug_0_4030.jpg
File: fruit_dataset/archive (1)/dataset/dataset/train/unripe banana/aug_0_5098.jpg
File: fruit_dataset/archive (1)/dataset/dataset/train/unripe banana/aug_0_1038.jpg
File: fruit_dataset/archive (1)/dataset/dataset/train/unripe banana/aug_0_5320.jpg
File: 

In [6]:
!pip install tensorflow


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Define the path to your dataset
train_dir = '/content/fruit_dataset/archive (1)/dataset/dataset/train'

# Image dimensions
img_size = 224  # Size for MobileNetV2

# Data Preprocessing: Rescaling and Augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2  # 20% of data for validation
)

# Validation data generator, using the split from training data
val_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_size, img_size),
    batch_size=32,
    class_mode='categorical',
    subset='training',  # Use training subset
)

val_generator = val_datagen.flow_from_directory(
    train_dir,
    target_size=(img_size, img_size),
    batch_size=32,
    class_mode='categorical',
    subset='validation',  # Use validation subset
)

# Load pre-trained MobileNetV2 model + top (classifier) layers
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(img_size, img_size, 3))

# Freeze the convolutional base (this helps avoid overfitting)
base_model.trainable = False

# Build the model by adding custom layers on top of MobileNetV2
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dropout(0.5),
    Dense(1024, activation='relu'),
    Dense(train_generator.num_classes, activation='softmax')
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    epochs=10,  # Start with 10 epochs (you can increase later)
    validation_data=val_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_steps=val_generator.samples // val_generator.batch_size,
)

# Save the model
model.save('fruit_shelf_life_model.h5')


Found 10932 images belonging to 9 classes.
Found 2727 images belonging to 9 classes.
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
341/341 ━━━━━━━━━━━━━━━━━━━━ 811s 2s/step - accuracy: 0.5728 - loss: 1.1649 - val_accuracy: 0.8938 - val_loss: 0.2962
Epoch 2/10
  1/341 ━━━━━━━━━━━━━━━━━━━━ 7:35 1s/step - accuracy: 0.7812 - loss: 0.4860

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


341/341 ━━━━━━━━━━━━━━━━━━━━ 140s 409ms/step - accuracy: 0.7812 - loss: 0.4860 - val_accuracy: 0.8934 - val_loss: 0.2952
Epoch 3/10
341/341 ━━━━━━━━━━━━━━━━━━━━ 841s 2s/step - accuracy: 0.8321 - loss: 0.4402 - val_accuracy: 0.9169 - val_loss: 0.2279
Epoch 4/10
341/341 ━━━━━━━━━━━━━━━━━━━━ 112s 324ms/step - accuracy: 0.7812 - loss: 0.4825 - val_accuracy: 0.9180 - val_loss: 0.2291
Epoch 5/10
341/341 ━━━━━━━━━━━━━━━━━━━━ 747s 2s/step - accuracy: 0.8578 - loss: 0.3605 - val_accuracy: 0.9261 - val_loss: 0.2005
Epoch 6/10
341/341 ━━━━━━━━━━━━━━━━━━━━ 114s 332ms/step - accuracy: 0.8750 - loss: 0.3778 - val_accuracy: 0.9268 - val_loss: 0.2014
Epoch 7/10
341/341 ━━━━━━━━━━━━━━━━━━━━ 679s 2s/step - accuracy: 0.8707 - loss: 0.3466 - val_accuracy: 0.9327 - val_loss: 0.1787
Epoch 8/10
341/341 ━━━━━━━━━━━━━━━━━━━━ 109s 316ms/step - accuracy: 0.9062 - loss: 0.2471 - val_accuracy: 0.9335 - val_loss: 0.1768
Epoch 9/10
341/341 ━━━━━━━━━━━━━━━━━━━━ 674s 2s/step - accuracy: 0.8804 - loss: 0.3212 - val_acc

In [33]:
model.save('Shelf_life_prediction.keras')

In [34]:
import os
print("Current Directory:", os.getcwd())


Current Directory: /content


In [35]:
print("Files in the current directory:")
print(os.listdir())


Files in the current directory:
['.config', 'fruit_dataset', 'kaggle.json', 'fruit_shelf_life_model.h5', 'Shelf_life_prediction.keras', 'kaggle (1).json', 'best_model.h5', 'fruit-ripeness-unripe-ripe-and-rotten.zip', 'fruit_dataset_clean', 'fruit_images', 'sample_data']


In [36]:
from tensorflow.keras.models import load_model

# Replace '/content/model.h5' with your model file's path
model = load_model('/content/Shelf_life_prediction.keras')
model.summary()  # Optional: Check the model summary to verify the architecture


/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 6 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1024)           │     1,311,744 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 9)              │         9,225 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,899,924 (18.69 MB)

 Trainable params: 1,320,969 (5.04 MB)

 Non-trainable params: 2,257,984 (8.61 MB)

 Optimizer params: 1,320,971 (5.04 MB)

In [37]:
from google.colab import files

# Replace with your model's file path
model_path = '/content/Shelf_life_prediction.keras'

# Download the file
files.download(model_path)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [40]:
# Change to the folder where your zip file is located
os.chdir('/content/fruit_dataset/archive (1)/dataset/dataset/train')


In [44]:
import os
import json
from google.colab import files

# Path to your dataset directory (change this if your dataset is in a different location)
dataset_path = '/content/fruit_dataset/archive (1)/dataset/dataset/train'

# Get the list of subfolders (these should correspond to the classes)
class_names = sorted(os.listdir(dataset_path))

# Verify the class names
print("Class names (sorted):", class_names)

# Create a dictionary with class names as keys and indices as values
class_dict = {class_name: index for index, class_name in enumerate(class_names)}

# Path to save the JSON file
json_path = '/content/fruit_dataset/archive (1)/dataset/dataset/train/classes.json'

# Save the class dictionary to a JSON file
with open(json_path, 'w') as f:
    json.dump(class_dict, f, indent=4)

# Download the classes.json file
files.download(json_path)

print(f"Classes saved and downloaded as {json_path}")


Class names (sorted): ['classes.json', 'freshapples', 'freshbanana', 'freshoranges', 'rottenapples', 'rottenbanana', 'rottenoranges', 'unripe apple', 'unripe banana', 'unripe orange']


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Classes saved and downloaded as /content/fruit_dataset/archive (1)/dataset/dataset/train/classes.json
